In [1]:
import sys 
sys.path.append('/Users/willknott/Desktop/DIS/coursework/pds/wdk24/src')

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

In [2]:
from generation import generate_from_total_pdf
from distributions import total_cdf
from hypothesis_test import signal_background_test

In [3]:
# True parameter values
true_params = {'f': 0.1, 'lam': 0.5, 'mu': 5.28, 'sigma': 0.018}

# Upper & Lower bound of M
alpha = 5
beta = 5.6

random_shifts = {
    'f': np.random.uniform(-0.09, 0.5), 
    'lam': np.random.uniform(-0.3, 1), 
    'mu': np.random.uniform(-0.25, 0.3), 
    'sigma': np.random.uniform(-0.01, 0.03)
}

# Creating starting parameters
starting_params = {}
for param in true_params:
    starting_params[param] = true_params[param] + random_shifts[param]
    starting_params[param] = float(f'{starting_params[param]:.4}')

In [4]:
n_datapoints = 40
Ns = np.logspace(1, 6, num=n_datapoints).astype(int)
n_init = 25
ns = np.array([n_init for _ in range(n_datapoints)])

all_data = [np.array([generate_from_total_pdf(N) for _ in range(n_init)]) for N in Ns]


In [5]:
def error_on_p(p, n):
    return np.sqrt((p*(1-p))/n)

In [6]:
Ps = []
P_errs = []
for i, group_i in enumerate(all_data):

    r_discoveries = 0
    for dataset in group_i:
        discovery, _, _ = signal_background_test(dataset=dataset, cdf=total_cdf, starting_params=starting_params)
        if discovery:
            r_discoveries += 1
    p = r_discoveries/ns[i]
    Ps.append(p)
    P_errs.append(error_on_p(p, ns[i]))

/Users/willknott/Desktop/DIS/coursework/pds/wdk24/src/distributions.py:115: RuntimeWarning: divide by zero encountered in scalar divide
  background_factor = 1/(expon.cdf(x=beta, scale=1/lam) - expon.cdf(x=alpha, scale=1/lam))
/Users/willknott/Desktop/DIS/coursework/pds/wdk24/src/distributions.py:118: RuntimeWarning: invalid value encountered in multiply
  background_cdf = background_factor*(expon.cdf(x=M, scale=1/lam) - expon.cdf(x=alpha, scale=1/lam))
/Users/willknott/Desktop/DIS/coursework/pds/wdk24/src/distributions.py:114: RuntimeWarning: divide by zero encountered in scalar divide
  signal_factor = 1/(norm.cdf(x=beta, loc=mu, scale=sigma) - norm.cdf(x=alpha, loc=mu, scale=sigma))
/Users/willknott/Desktop/DIS/coursework/pds/wdk24/src/distributions.py:117: RuntimeWarning: invalid value encountered in multiply
  signal_cdf = signal_factor*(norm.cdf(x=M, loc=mu, scale=sigma) - norm.cdf(x=alpha, loc=mu, scale=sigma))


In [ ]:
print(Ps)
print(P_errs)

[0.0, 0.0, 0.04, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.0, 0.0, 0.039191835884530846, 0.039191835884530846, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06499230723708768, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# successes = []
# for N_i in Ns:
#     n_discoveries = 0
#     for _ in range(n_init):
#         dataset = generate_from_total_pdf(N_i)

#         discovery, _, _ = signal_background_test(dataset=dataset, cdf=total_cdf, starting_params=starting_params)

#         if discovery:
#             n_discoveries += 1
#     successes.append(n_discoveries)

dict(zip(Ns, successes))

# Finding `N_90`